##Import Data and Python Packages

In [442]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score
from scipy import stats

sns.set()
stats.chisqprob = lambda chiq, df:stats.chi2.sf(chisq, sf)

In [385]:
raw_data = pd.read_csv("DSChallenge.csv")

In [386]:
raw_data.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,ID1,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,ID2,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,ID3,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,ID4,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,ID5,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [387]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


In [388]:
raw_data=raw_data.rename({"Loan_ID": "loan_id", "ApplicantIncome":"applicant_income", 
                 "CoApplicantIncome":"co_applicant_income", "LoanAmount":"loan_amount", 
                 "Loan_Amount_Term":"loan_amount_term", "Credit_History":"credit_history",
                 "Property_Area":"property_area", "Loan_Status":"loan_status"},  axis = 1)

In [389]:
columns=[]
for col in raw_data.columns:
  col = col.lower()
  columns.append(col) 

In [390]:
raw_data.columns = columns
raw_data.columns

Index(['loan_id', 'gender', 'married', 'dependents', 'education',
       'self_employed', 'applicant_income', 'coapplicantincome', 'loan_amount',
       'loan_amount_term', 'credit_history', 'property_area', 'loan_status'],
      dtype='object')

##Preprocessing

In [391]:
raw_data.describe(include = 'all')

,loan_id,gender,married,dependents,education,self_employed,applicant_income,coapplicantincome,loan_amount,loan_amount_term,credit_history,property_area,loan_status
count,614,601,611,599,614,582,614.000000,614.000000,592.000000,600.00000,564.000000,614,614
unique,614,2,2,4,2,2,NaN,NaN,NaN,NaN,NaN,3,2
top,ID342,Male,Yes,0,Graduate,No,NaN,NaN,NaN,NaN,NaN,Semiurban,Y
freq,1,489,398,345,480,500,NaN,NaN,NaN,NaN,NaN,233,422
mean,NaN,NaN,NaN,NaN,NaN,NaN,5403.459283,1621.245798,146.412162,342.00000,0.842199,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,6109.041673,2926.248369,85.587325,65.12041,0.364878,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,150.000000,0.000000,9.000000,12.00000,0.000000,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,2877.500000,0.000000,100.000000,360.00000,1.000000,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,3812.500000,1188.500000,128.000000,360.00000,1.000000,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,5795.000000,2297.250000,168.000000,360.00000,1.000000,NaN,NaN


In [396]:
data = raw_data.drop(['gender', 'loan_id'], axis=1)

1. Each variable has a different number of observations, which implies that there are some missing values 
2. *gender* has 489 male entries, which is almost 80% of the data. This feature probably won't be useful


In [397]:
data.describe(include='all')

,married,dependents,education,self_employed,applicant_income,coapplicantincome,loan_amount,loan_amount_term,credit_history,property_area,loan_status
count,611,599,614,582,614.000000,614.000000,592.000000,600.00000,564.000000,614,614
unique,2,4,2,2,NaN,NaN,NaN,NaN,NaN,3,2
top,Yes,0,Graduate,No,NaN,NaN,NaN,NaN,NaN,Semiurban,Y
freq,398,345,480,500,NaN,NaN,NaN,NaN,NaN,233,422
mean,NaN,NaN,NaN,NaN,5403.459283,1621.245798,146.412162,342.00000,0.842199,NaN,NaN
std,NaN,NaN,NaN,NaN,6109.041673,2926.248369,85.587325,65.12041,0.364878,NaN,NaN
min,NaN,NaN,NaN,NaN,150.000000,0.000000,9.000000,12.00000,0.000000,NaN,NaN
25%,NaN,NaN,NaN,NaN,2877.500000,0.000000,100.000000,360.00000,1.000000,NaN,NaN
50%,NaN,NaN,NaN,NaN,3812.500000,1188.500000,128.000000,360.00000,1.000000,NaN,NaN
75%,NaN,NaN,NaN,NaN,5795.000000,2297.250000,168.000000,360.00000,1.000000,NaN,NaN


###Dealing With Missing Values

In [398]:
data.isnull().sum()

married               3
dependents           15
education             0
self_employed        32
applicant_income      0
coapplicantincome     0
loan_amount          22
loan_amount_term     14
credit_history       50
property_area         0
loan_status           0
dtype: int64

In [400]:
married=data[data["married"].isnull()][["dependents", "self_employed"]]
married

,dependents,self_employed
104,NaN,No
228,NaN,No
435,NaN,No


In [402]:
data["married"] = data["married"].replace(np.nan, "No")
data["married"].isnull().sum()

0

In [403]:
data["self_employed"]=data["self_employed"].replace(np.nan, "No")
data["self_employed"].isnull().sum()

0

In [404]:
data["loan_amount"].fillna(data["loan_amount"].median(skipna=True), inplace=True)
data["loan_amount"].isnull().sum()

0

In [405]:
data["loan_amount_term"].fillna(data["loan_amount_term"].median(skipna=True), inplace=True)
data["loan_amount_term"].isnull().sum()

0

In [406]:
data["credit_history"].fillna(data["credit_history"].median(skipna=True), inplace=True)
data["credit_history"].isnull().sum()

0

In [407]:
data_no_mv = data.dropna(axis=0)

In [408]:
data_no_mv.isnull().sum()

married              0
dependents           0
education            0
self_employed        0
applicant_income     0
coapplicantincome    0
loan_amount          0
loan_amount_term     0
credit_history       0
property_area        0
loan_status          0
dtype: int64

In [409]:
data_no_mv.describe(include='all')

,married,dependents,education,self_employed,applicant_income,coapplicantincome,loan_amount,loan_amount_term,credit_history,property_area,loan_status
count,599,599,599,599,599.000000,599.000000,599.000000,599.000000,599.000000,599,599
unique,2,4,2,2,NaN,NaN,NaN,NaN,NaN,3,2
top,Yes,0,Graduate,No,NaN,NaN,NaN,NaN,NaN,Semiurban,Y
freq,390,345,469,519,NaN,NaN,NaN,NaN,NaN,227,413
mean,NaN,NaN,NaN,NaN,5408.864775,1634.896361,146.188648,342.070117,0.859766,NaN,NaN
std,NaN,NaN,NaN,NaN,6168.473138,2953.144575,84.954303,64.221516,0.347520,NaN,NaN
min,NaN,NaN,NaN,NaN,150.000000,0.000000,9.000000,12.000000,0.000000,NaN,NaN
25%,NaN,NaN,NaN,NaN,2853.000000,0.000000,100.000000,360.000000,1.000000,NaN,NaN
50%,NaN,NaN,NaN,NaN,3775.000000,1213.000000,128.000000,360.000000,1.000000,NaN,NaN
75%,NaN,NaN,NaN,NaN,5816.500000,2304.000000,165.500000,360.000000,1.000000,NaN,NaN


###Encoding Categorical Data

In [410]:
obj_df=data_no_mv.select_dtypes(include=['object']).copy()
obj_df.head()

,married,dependents,education,self_employed,property_area,loan_status
0,No,0,Graduate,No,Urban,Y
1,Yes,1,Graduate,No,Rural,N
2,Yes,0,Graduate,Yes,Urban,Y
3,Yes,0,Not Graduate,No,Urban,Y
4,No,0,Graduate,No,Urban,Y


In [411]:
obj_df["married"].value_counts()

Yes    390
No     209
Name: married, dtype: int64

In [412]:
obj_df["dependents"].value_counts()

0     345
1     102
2     101
3+     51
Name: dependents, dtype: int64

In [413]:
obj_df["education"].value_counts()

Graduate        469
Not Graduate    130
Name: education, dtype: int64

In [414]:
obj_df["self_employed"].value_counts()

No     519
Yes     80
Name: self_employed, dtype: int64

In [415]:
obj_df["property_area"].value_counts()

Semiurban    227
Urban        195
Rural        177
Name: property_area, dtype: int64

In [416]:
obj_df["loan_status"].value_counts()

Y    413
N    186
Name: loan_status, dtype: int64

In [417]:
cleanup_nums = {"married":     {"Yes": 1, "No": 0},
                "dependents": {"0": 0, "1": 1, "2": 2, "3+": 3},
                "education" : {"Graduate": 1, "Not Graduate" : 0},
                "self_employed": {"Yes" : 1, "No" : 0},
                "property_area": {"Rural" : 0, "Semiurban" :1, "Urban":2},
                "loan_status": {"N" : 0, "Y":1}
               }

In [418]:
obj_df = obj_df.replace(cleanup_nums)
obj_df.head()

,married,dependents,education,self_employed,property_area,loan_status
0,0,0,1,0,2,1
1,1,1,1,0,0,0
2,1,0,1,1,2,1
3,1,0,0,0,2,1
4,0,0,1,0,2,1


In [419]:
data_no_mv[['married', 'dependents', 'education', 'self_employed', 'property_area', 'loan_status']]=obj_df

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [420]:
data_no_mv.head()

,married,dependents,education,self_employed,applicant_income,coapplicantincome,loan_amount,loan_amount_term,credit_history,property_area,loan_status
0,0,0,1,0,5849,0.0,128.0,360.0,1.0,2,1
1,1,1,1,0,4583,1508.0,128.0,360.0,1.0,0,0
2,1,0,1,1,3000,0.0,66.0,360.0,1.0,2,1
3,1,0,0,0,2583,2358.0,120.0,360.0,1.0,2,1
4,0,0,1,0,6000,0.0,141.0,360.0,1.0,2,1


In [421]:
data_no_mv.columns

Index(['married', 'dependents', 'education', 'self_employed',
       'applicant_income', 'coapplicantincome', 'loan_amount',
       'loan_amount_term', 'credit_history', 'property_area', 'loan_status'],
      dtype='object')

In [422]:
X=data_no_mv.iloc[:, :-1].values

In [423]:
y = data_no_mv.iloc[:,-1].values

In [424]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

###Feature Scaling

In [425]:
from sklearn.preprocessing import StandardScaler
X_train[:, 4:8]

array([[1378.00, 1881.00, 167.00, 360.00],
       [3166.00, 0.00, 36.00, 360.00],
       [3875.00, 0.00, 67.00, 360.00],
       ...,
       [3500.00, 0.00, 81.00, 300.00],
       [5819.00, 5000.00, 120.00, 360.00],
       [3667.00, 1459.00, 144.00, 360.00]])

##Training the Model

###Logistic Regression (Logit function)

In [426]:
x = sm.add_constant(X_train)
reg_log = sm.Logit(y_train, x)
results_log=reg_log.fit()

Optimization terminated successfully.
         Current function value: 0.465782
         Iterations 7


In [427]:
results_log.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                  479
Model:                          Logit   Df Residuals:                      468
Method:                           MLE   Df Model:                           10
Date:                Sat, 17 Jul 2021   Pseudo R-squ.:                  0.2425
Time:                        01:37:44   Log-Likelihood:                -223.11
converged:                       True   LL-Null:                       -294.53
Covariance Type:            nonrobust   LLR p-value:                 1.106e-25
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.8963      0.923     -3.139      0.002      -4.705      -1.088
x1             0.5416      0.265      2.044      0.041       0.022       1.061
x2            -0.0120      0.132     -0.091      0.928      -0.272       0.248
x3             0.3647      0.295      1.238      0.216      -0.213       0.942
x4            -0.0900      0.351     -0.256      0.798      -0.778       0.598
x5          1.816e-05   2.47e-05      0.736      0.462   -3.02e-05    6.65e-05
x6         -3.841e-05   4.02e-05     -0.957      0.339      -0.000    4.03e-05
x7            -0.0008      0.002     -0.449      0.653      -0.004       0.003
x8            -0.0009      0.002     -0.419      0.675      -0.005       0.003
x9             3.9284      0.493      7.969      0.000       2.962       4.895
x10            0.0949      0.155      0.611      0.541      -0.210       0.400
==============================================================================
"""

In [428]:
np.set_printoptions(formatter={'float': lambda x: '{0:0.2f}'.format(x)})
results_log.predict()

array([0.83, 0.85, 0.86, 0.77, 0.71, 0.85, 0.73, 0.67, 0.06, 0.83, 0.83,
       0.85, 0.75, 0.88, 0.85, 0.38, 0.78, 0.77, 0.78, 0.83, 0.72, 0.81,
       0.79, 0.76, 0.85, 0.82, 0.86, 0.73, 0.62, 0.83, 0.80, 0.83, 0.83,
       0.78, 0.68, 0.81, 0.87, 0.87, 0.84, 0.81, 0.76, 0.78, 0.78, 0.08,
       0.07, 0.82, 0.74, 0.86, 0.77, 0.88, 0.83, 0.84, 0.83, 0.84, 0.89,
       0.06, 0.81, 0.06, 0.71, 0.78, 0.08, 0.86, 0.81, 0.78, 0.75, 0.77,
       0.78, 0.73, 0.69, 0.84, 0.77, 0.86, 0.83, 0.76, 0.84, 0.71, 0.84,
       0.86, 0.78, 0.75, 0.76, 0.04, 0.75, 0.79, 0.05, 0.67, 0.06, 0.81,
       0.82, 0.81, 0.79, 0.79, 0.82, 0.87, 0.80, 0.76, 0.84, 0.75, 0.83,
       0.06, 0.85, 0.77, 0.79, 0.75, 0.86, 0.78, 0.79, 0.08, 0.83, 0.85,
       0.73, 0.09, 0.07, 0.81, 0.86, 0.85, 0.84, 0.87, 0.83, 0.78, 0.90,
       0.71, 0.05, 0.75, 0.77, 0.06, 0.07, 0.71, 0.76, 0.85, 0.75, 0.76,
       0.87, 0.75, 0.84, 0.76, 0.81, 0.83, 0.08, 0.85, 0.73, 0.85, 0.78,
       0.10, 0.70, 0.66, 0.85, 0.85, 0.77, 0.70, 0.

In [429]:
np.array(data_no_mv['loan_status'])

array([1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1,
       0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1,
       0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1,
       1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0,

In [430]:
results_log.pred_table()

array([[62.00, 84.00],
       [5.00, 328.00]])

In [431]:
cm_df = pd.DataFrame(results_log.pred_table())
cm_df.columns = ['Predicted 0', 'Predicted 1']
cm_df = cm_df.rename(index={0:'Actual 0', 1: 'Actual 1'})
cm_df

,Predicted 0,Predicted 1
Actual 0,62.0,84.0
Actual 1,5.0,328.0


In [432]:
cm = np.array(cm_df)
accuracy_score = (cm[0,0]+cm[1,1])/cm.sum()
print("Accuracy score: %.2f" %(accuracy_score*100), "%")

Accuracy score: 81.42 %


###Testing the Model

In [433]:
x_test = sm.add_constant(X_test)
reg_log = sm.Logit(y_test, x_test)

In [434]:
def confusion_matrix(data,actual_values,model):
    
        pred_values = model.predict(data)
        bins=np.array([0,0.5,1])
        cm = np.histogram2d(actual_values, pred_values, bins=bins)[0]
        accuracy = (cm[0,0]+cm[1,1])/cm.sum()
        return cm, accuracy

In [435]:
cm = confusion_matrix(x_test,y_test,results_log)
cm

(array([[17.00, 23.00],
        [2.00, 78.00]]), 0.7916666666666666)

In [436]:
cm_df = pd.DataFrame(cm[0])
cm_df.columns = ['Predicted 0','Predicted 1']
cm_df = cm_df.rename(index={0: 'Actual 0',1:'Actual 1'})
cm_df

,Predicted 0,Predicted 1
Actual 0,17.0,23.0
Actual 1,2.0,78.0


In [437]:
print ('Missclassification rate: '+str((1+1)/19))

Missclassification rate: 0.10526315789473684


###Model evaluation based on K-fold-cross validation

In [438]:
classifier = LogisticRegression()
classifier.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [444]:
y_pred = classifier.predict(X_test)
confusion_matrix(y_test, y_pred)
a_s=accuracy_score(y_test, y_pred)
a_s*=100
print("Accuracy score: %.2f"%a_s)

Accuracy score: 79.17


In [445]:
accuracies = cross_val_score(estimator = classifier, X=X_train, y=y_train, cv=10)
print("Accuracy  (K-fold-cross validation): {:.2f}%".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f}%".format(accuracies.std()*100))

Accuracy  (K-fold-cross validation): 80.80%
Standard Deviation: 6.29%


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
